In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy.special import expit, logit
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale

np.random.seed(1)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)

# Prepare the data

In [2]:
# load
df = pd.read_pickle('../data/process/schools2017.pkl')

# select columns
df = df[[
    'Charter School?',
    'Percent Asian',
    'Percent Black',
    'Percent Hispanic',
    'Percent Other',
    'Percent English Language Learners',
    'Percent Students with Disabilities',
    'Economic Need Index',
        
    'Mean Scale Score - ELA',
    '% Level 2 - ELA',
    '% Level 3 - ELA',
    '% Level 4 - ELA',
    'Mean Scale Score - Math',
    '% Level 2 - Math',
    '% Level 3 - Math',
    '% Level 4 - Math',
    
    '# Students in HS Admissions',
    '# SHSAT Testers',
    '% SHSAT Testers',
]].copy()
print(df.shape[0], "schools")

# drop schools with missing test data
df = df[df.loc[:, 'Mean Scale Score - ELA':'% Level 4 - Math'].notnull().all(axis=1)]
print(df.shape[0], "schools after dropping missing test data")

# schools with 0-5 SHSAT testers have this value set to NaN
applicantsok = df['# SHSAT Testers'].notnull()

# convert percentages to the (0, 1) range
bad_pct_c = [
    '% Level 2 - ELA',
    '% Level 3 - ELA',
    '% Level 4 - ELA',
    '% Level 2 - Math',
    '% Level 3 - Math',
    '% Level 4 - Math',
]
df.loc[:, bad_pct_c] = df.loc[:, bad_pct_c] / 100.0

# standardize score columns (ease of interpretation + algorithm stability)
score_c = ['Mean Scale Score - ELA', 'Mean Scale Score - Math']
df.loc[:, score_c] = df.loc[:, score_c].apply(scale)

594 schools
588 schools after dropping missing test data


# Create model

This time let's use only one model, to simplify the things.

In [3]:
base_df = df[[  # explanatory variables
    'Charter School?',
    'Percent Asian',
    'Percent Black',
    'Percent Hispanic',
    'Percent Other',
    'Percent English Language Learners',
    'Percent Students with Disabilities',
    'Economic Need Index',
    
    'Mean Scale Score - ELA',
    '% Level 2 - ELA',
    '% Level 3 - ELA',
    '% Level 4 - ELA',
    'Mean Scale Score - Math',
    '% Level 2 - Math',
    '% Level 3 - Math',
    '% Level 4 - Math',
]]

n_components = 8
pca = PCA(n_components)
transformed = pca.fit_transform(base_df)
transformed = pd.DataFrame(transformed, index=base_df.index, columns=["PC{}".format(i+1) for i in range(n_components)])
transformed.head()

inputs = transformed
inputs.insert(0, 'Constant', 1.0)
inputs.head()

,Constant,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8
DBN,,,,,,,,,
01M034,1.0,-1.069606,-0.153384,0.039633,-0.129827,-0.134514,-0.037738,0.079893,0.094137
01M140,1.0,-0.946293,-0.225142,0.350088,-0.097681,-0.160674,-0.081404,0.024806,0.132633
01M184,1.0,2.162533,-0.368097,0.142796,0.366878,0.171430,0.099375,0.412524,-0.019146
01M188,1.0,-0.919823,-0.084855,0.284476,0.167532,-0.252479,-0.016074,0.014955,0.055994
01M301,1.0,-0.689744,-0.184419,-0.173224,-0.227307,-0.053143,-0.068751,0.068501,0.020560


In [13]:
#data
inputs_fit = inputs[applicantsok]
outputs_fit = logit(df['% SHSAT Testers'][applicantsok])
inputs_predict = inputs

# fit
model = sm.RLM(outputs_fit, inputs_fit, M=sm.robust.norms.HuberT())
results = model.fit()

# predict
predictions = model.predict(results.params, exog=inputs_predict)
predictions = pd.Series(predictions, index=inputs_predict.index)
predictions.name = 'Predictions'

# score
from sklearn.metrics import mean_squared_error, mean_absolute_error, median_absolute_error, r2_score
print(mean_squared_error(outputs_fit, predictions[applicantsok]))
print(median_absolute_error(outputs_fit, predictions[applicantsok]))

0.45073380581294925
0.35800549417912975


These are nice values... It can be made better with the chronic absenteeism, but, save this for later.

In [5]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                    Robust linear Model Regression Results                    
==============================================================================
Dep. Variable:        % SHSAT Testers   No. Observations:                  533
Model:                            RLM   Df Residuals:                      524
Method:                          IRLS   Df Model:                            8
Norm:                          HuberT                                         
Scale Est.:                       mad                                         
Cov Type:                          H1                                         
Date:                Fri, 03 Aug 2018                                         
Time:                        00:21:48                                         
No. Iterations:                    25                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Constant      -0.9134      0.025    -36.655      0.000      -0.962      -0.865
PC1            0.5343      0.018     30.237      0.000       0.500       0.569
PC2            0.1009      0.057      1.764      0.078      -0.011       0.213
PC3           -0.1769      0.073     -2.430      0.015      -0.320      -0.034
PC4            0.2623      0.095      2.760      0.006       0.076       0.449
PC5            0.6188      0.141      4.398      0.000       0.343       0.895
PC6            0.9718      0.170      5.720      0.000       0.639       1.305
PC7            0.6771      0.212      3.196      0.001       0.262       1.092
PC8           -0.4701      0.262     -1.792      0.073      -0.984       0.044
==============================================================================

If the model instance has been used for another fit with different fit
parameters, then the fit options might not be the correct ones anymore .
"""

Can't see the null hypothesis .

# Arrange results

In [6]:
def from_counts(shsat_counts, hs_counts):
    return pd.DataFrame({
        'logit': logit(shsat_counts / hs_counts),
        'pct': shsat_counts / hs_counts,
        'cnt': shsat_counts,
    })

def from_logits(shsat_logits, hs_counts):
    return pd.DataFrame({
        'logit': shsat_logits,
        'pct': expit(shsat_logits),
        'cnt': expit(shsat_logits) * hs_counts,
    })

In [7]:
# actual values  ---

# schools with 0 to 5 applicants
hs_counts = df[~applicantsok]['# Students in HS Admissions']
min_v = from_counts(0, hs_counts)
max_v = from_counts(5, hs_counts)

# schools with 6 or more applicants
shsat_counts = df[applicantsok]['# SHSAT Testers']
hs_counts = df[applicantsok]['# Students in HS Admissions']
pontual_v = from_counts(shsat_counts, hs_counts)


# expected values  ---

shsat_logits = predictions
hs_counts = df['# Students in HS Admissions']
expected_v = from_logits(shsat_logits, hs_counts)


# differences  ---

min_diff = (min_v - expected_v).dropna()
max_diff = (max_v - expected_v).dropna()
pontual_diff = (pontual_v - expected_v).dropna()


# join everything  ---

# actual
actual = pd.DataFrame({
    'Actual Cnt (min)': pd.concat([min_v, pontual_v])['cnt'],
    'Actual Cnt (max)': pd.concat([max_v, pontual_v])['cnt'],
    'Actual Pct (min)': pd.concat([min_v, pontual_v])['pct'],
    'Actual Pct (max)': pd.concat([max_v, pontual_v])['pct'],
})

# expected
expected = pd.DataFrame({
    'Expected Cnt': expected_v['cnt'],
    'Expected Pct': expected_v['pct'],
})

# diff
difference = pd.DataFrame({
    'Diff Cnt (min)': pd.concat([min_diff, pontual_diff])['cnt'],
    'Diff Cnt (max)': pd.concat([max_diff, pontual_diff])['cnt'],
    'Diff Pct (min)': pd.concat([min_diff, pontual_diff])['pct'],
    'Diff Pct (max)': pd.concat([max_diff, pontual_diff])['pct'],
    
    'Chance Multiplier': np.exp(pd.concat([max_diff, pontual_diff])['logit']),  # note - max diff
})

# all
everything = actual.join(expected).join(difference)
everything = everything.sort_index()
everything.head()

,Actual Cnt (min),Actual Cnt (max),Actual Pct (min),Actual Pct (max),Expected Cnt,Expected Pct,Diff Cnt (min),Diff Cnt (max),Diff Pct (min),Diff Pct (max),Chance Multiplier
DBN,,,,,,,,,,,
01M034,6.0,6.0,0.103448,0.103448,9.331878,0.160894,-3.331878,-3.331878,-0.057446,-0.057446,0.601760
01M140,6.0,6.0,0.089552,0.089552,9.890076,0.147613,-3.890076,-3.890076,-0.058061,-0.058061,0.567980
01M184,67.0,67.0,0.761364,0.761364,60.091081,0.682853,6.908919,6.908919,0.078510,0.078510,1.481796
01M188,0.0,5.0,0.000000,0.084746,9.845694,0.166876,-9.845694,-4.845694,-0.166876,-0.082130,0.462266
01M301,11.0,11.0,0.215686,0.215686,10.148903,0.198998,0.851097,0.851097,0.016688,0.016688,1.106923


In [8]:
everything.sort_values('Chance Multiplier').head()

,Actual Cnt (min),Actual Cnt (max),Actual Pct (min),Actual Pct (max),Expected Cnt,Expected Pct,Diff Cnt (min),Diff Cnt (max),Diff Pct (min),Diff Pct (max),Chance Multiplier
DBN,,,,,,,,,,,
02M408,9.0,9.0,0.272727,0.272727,24.514137,0.742853,-15.514137,-15.514137,-0.470125,-0.470125,0.129811
84M353,0.0,5.0,0.000000,0.049505,23.059873,0.228316,-23.059873,-18.059873,-0.228316,-0.178811,0.176037
84M204,0.0,5.0,0.000000,0.060976,21.984973,0.268109,-21.984973,-16.984973,-0.268109,-0.207134,0.177261
84M336,15.0,15.0,0.159574,0.159574,46.826883,0.498158,-31.826883,-31.826883,-0.338584,-0.338584,0.191277
09X117,0.0,5.0,0.000000,0.028571,21.948529,0.125420,-21.948529,-16.948529,-0.125420,-0.096849,0.205094


# Usage example

PASSNYC wants to find a school where it has the biggest chance of bringing new students to sit for the SHSAT.

In [9]:
# generate a simple table for visualization first

school_info = pd.read_pickle('../data/process/schools2017.pkl')
school_info = school_info[[
    'Borough',
    
    'Percent Hispanic',
    'Percent Black',
    'Percent White',
    'Percent Asian',
    
    'Economic Need Index',
    'Percent English Language Learners',
    'Percent Students with Disabilities',
    'Percent of Students Chronically Absent',
    
    '% Level 4 - ELA',
    '% Level 4 - Math',
    
    '# Students in HS Admissions',
]]

school_info.columns = [
    'Borough',
    
    'Hispanic',
    'Black',
    'White',
    'Asian',
    
    'Economic Need Index',
    'ELL',
    'Special Education',
    'Chronically Absent',
    
    'ELA 4s',
    'Math 4s',
    
    'HS Admissions',
]

school_info['ELA 4s'] /= 100.0
school_info['Math 4s'] /= 100.0

In [10]:
def join_range(x1, x2):
    x1, x2 = sorted([x1, x2])
    x1 = np.round(x1).astype(int)
    x2 = np.round(x2).astype(int)
    if x1 == x2:
        return "{}".format(x1)
    else:
        return "{}-{}".format(x1, x2)
    
expected_cnt = everything.apply(lambda x: join_range(x['Expected Cnt'], x['Expected Cnt']), axis=1)
actual_cnt = everything.apply(lambda x: join_range(x['Actual Cnt (min)'], x['Actual Cnt (max)']), axis=1)
diff_cnt = everything.apply(lambda x: join_range(-x['Diff Cnt (min)'], -x['Diff Cnt (max)']), axis=1)

school_info['Testers (Exp)'] = expected_cnt
school_info['Testers (Actual)'] = actual_cnt
school_info['Testers (Diff)'] = diff_cnt

In [11]:
index = everything.sort_values('Chance Multiplier').index
school_info = school_info.reindex(index)

In [12]:
format_dict = {
    'Hispanic': '{:.0%}',
    'Black': '{:.0%}',
    'White': '{:.0%}',
    'Asian': '{:.0%}',
    
    'Economic Need Index': '{:.2f}',
    'ELL': '{:.0%}',
    'Special Education': '{:.0%}',
    'Chronically Absent': '{:.0%}',
    
    'ELA 4s': '{:.0%}',
    'Math 4s': '{:.0%}',
    
    'HS Admissions': '{:.0f}',
}
school_info.head(25).style.format(format_dict)

,Borough,Hispanic,Black,White,Asian,Economic Need Index,ELL,Special Education,Chronically Absent,ELA 4s,Math 4s,HS Admissions,Testers (Exp),Testers (Actual),Testers (Diff)
DBN,,,,,,,,,,,,,,,
02M408,manhattan,18%,17%,45%,10%,0.20,0%,12%,6%,75%,44%,33,25,9,16
84M353,manhattan,96%,4%,0%,0%,0.83,15%,14%,8%,5%,2%,101,23,0-5,18-23
84M204,manhattan,23%,72%,0%,2%,0.74,2%,24%,28%,5%,2%,82,22,0-5,17-22
84M336,manhattan,68%,30%,1%,0%,0.79,14%,23%,16%,21%,40%,94,47,15,32
09X117,bronx,79%,18%,1%,2%,0.90,40%,28%,25%,1%,0%,175,22,0-5,17-22
84X482,bronx,64%,32%,2%,1%,0.84,12%,25%,22%,3%,5%,110,24,7,17
09X004,bronx,42%,55%,1%,1%,0.91,13%,27%,50%,15%,15%,62,20,7,13
07X224,bronx,73%,25%,0%,0%,0.90,22%,29%,39%,0%,0%,129,16,0-5,11-16
84M386,manhattan,28%,66%,2%,1%,0.69,4%,23%,nan%,24%,44%,66,40,20,20


Use names from first notebooks